# Introduction : 

## This Python code sets up a Tic-Tac-Toe game where players can challenge a computer. The computer’s difficulty can be adjusted from ‘very easy’ to ‘very hard’. The game includes functions for showing the board, making moves, and checking for a winner. An advanced technique called Minimax with Alpha-Beta pruning helps the computer decide its moves, making the game interactive and fun for the player.

In [1]:
import random


# Maps difficulty levels to their corresponding depth values for AI moves.
DIFFICULTY_LEVELS = {
    'very easy': 0,
    'easy': 2,
    'medium': 4,
    'hard': 6,
    'very hard': 8
}

def display_board(board):
    """Displays the current state of the board."""
    print("\n")
    print(f" {board[0]} | {board[1]} | {board[2]}")
    print("---+---+---")
    print(f" {board[3]} | {board[4]} | {board[5]}")
    print("---+---+---")
    print(f" {board[6]} | {board[7]} | {board[8]}")
    print("\n")

def check_winner(board, mark):
    """Checks if the given mark has won the game."""
    win_conditions = [
        [board[0], board[1], board[2]],
        [board[3], board[4], board[5]],
        [board[6], board[7], board[8]],
        [board[0], board[3], board[6]],
        [board[1], board[4], board[7]],
        [board[2], board[5], board[8]],
        [board[0], board[4], board[8]],
        [board[2], board[4], board[6]],
    ]
    return [mark, mark, mark] in win_conditions

def make_move(board, mark, position):
    """Places a mark on the board at the given position."""
    board[position] = mark

def get_empty_positions(board):
    """Returns a list of empty positions on the board."""
    return [i for i, x in enumerate(board) if x == " "]

def player_move(board):
    """Handles the player's move."""
    while True:
        try:
            position = int(input("Choose your move (1-9): ")) - 1
            if position in get_empty_positions(board):
                return position
            else:
                print("Invalid move. The position is either taken or out of range. Try again.")
        except ValueError:
            print("Invalid input. Enter a number between 1 and 9.")

def ai_move(board, level):
    """Determines the AI's move based on the difficulty level."""
    if level == 'very easy':
        return random.choice(get_empty_positions(board))
    else:
        max_depth = DIFFICULTY_LEVELS[level]
        return optimal_move_alpha_beta(board, "O", max_depth)

def heuristic(board, mark):
    """Evaluates the board state and returns a heuristic score."""
    opponent_mark = "X" if mark == "O" else "O"
    if check_winner(board, mark):
        return 10
    elif check_winner(board, opponent_mark):
        return -10
    else:
        return 0

def minimax_alpha_beta(board, depth, is_maximizing, mark, max_depth, alpha, beta):
    """Minimax algorithm with alpha-beta pruning."""
    opponent_mark = "X" if mark == "O" else "O"
    score = heuristic(board, mark)
    if score == 10 or score == -10:
        return score
    if not get_empty_positions(board):
        return 0
    if depth >= max_depth:
        return heuristic(board, mark)

    if is_maximizing:
        best_score = -float('inf')
        for position in get_empty_positions(board):
            board[position] = mark
            score = minimax_alpha_beta(board, depth + 1, False, mark, max_depth, alpha, beta)
            board[position] = " "
            best_score = max(score, best_score)
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break
        return best_score
    else:
        best_score = float('inf')
        for position in get_empty_positions(board):
            board[position] = opponent_mark
            score = minimax_alpha_beta(board, depth + 1, True, mark, max_depth, alpha, beta)
            board[position] = " "
            best_score = min(score, best_score)
            beta = min(beta, best_score)
            if beta <= alpha:
                break
        return best_score

def optimal_move_alpha_beta(board, mark, max_depth):
    """Finds the optimal move using the minimax algorithm with alpha-beta pruning."""
    best_score = -float('inf')
    best_move = None
    for position in get_empty_positions(board):
        board[position] = mark
        score = minimax_alpha_beta(board, 0, False, mark, max_depth, -float('inf'), float('inf'))
        board[position] = " "
        if score > best_score:
            best_score = score
            best_move = position
    return best_move

def play_game(level, game_number):
    """Runs the game loop."""
    board = [" "] * 9
    current_turn = "Player"
    blunders = []
    move_count = 0
    
    while " " in board:
        display_board(board)
        if current_turn == "Player":
            position = player_move(board)
            make_move(board, "X", position)
            move_count += 1
            if check_winner(board, "X"):
                display_board(board)
                print("You win!")
                return "Player", blunders
            current_turn = "AI"
        else:
            print("\nAI is making a move...\n")
            position = ai_move(board, level)
            make_move(board, "O", position)
            if check_winner(board, "O"):
                display_board(board)
                print("AI wins!")
                blunders.append((game_number, move_count))
                return "AI", blunders
            current_turn = "Player"
    display_board(board)
    print("It's a tie!")
    return "Tie", blunders

def main():
    """Main function to run the game."""
    print("Welcome to Tic-Tac-Toe!")
    print("You will play as 'X' and the AI will play as 'O'.")
    user_name = input("Enter your name: ")
    game_stats = {"AI": 0, user_name: 0, "Tie": 0}
    cumulative_blunders = []
    game_number = 0
    while True:
        level = input("Choose difficulty (very easy, easy, medium, hard, very hard): ").lower()
        while level not in DIFFICULTY_LEVELS:
            level = input("Invalid choice. Choose difficulty (very easy, easy, medium, hard, very hard): ").lower()
        game_number += 1
        result, blunders = play_game(level, game_number)
        if result == "Player":
            game_stats[user_name] += 1
        else:
            game_stats[result] += 1
        cumulative_blunders.extend(blunders)
        
        cont = input("Do you want to play again? (yes/no): ").lower()
        if cont != 'yes':
            print("\nGame Report:")
            print(f"AI: {game_stats['AI']} | {user_name}: {game_stats[user_name]} | Tie: {game_stats['Tie']}")
            if game_stats['AI'] > 0:
                print("Blunders Report:")
                if len(cumulative_blunders) > 0:
                    for i, (game, move) in enumerate(cumulative_blunders):
                        print(f"Blunder {i + 1}: Game {game}, Move {move}")
                else:
                    print("No blunders; you are a champion!")
            else:
                print("No blunders; you are a champion!")
            break

if __name__ == "__main__":
    main()

Welcome to Tic-Tac-Toe!
You will play as 'X' and the AI will play as 'O'.
Enter your name: abc
Choose difficulty (very easy, easy, medium, hard, very hard): easy


   |   |  
---+---+---
   |   |  
---+---+---
   |   |  


Choose your move (1-9): 1


 X |   |  
---+---+---
   |   |  
---+---+---
   |   |  



AI is making a move...



 X | O |  
---+---+---
   |   |  
---+---+---
   |   |  


Choose your move (1-9): 4


 X | O |  
---+---+---
 X |   |  
---+---+---
   |   |  



AI is making a move...



 X | O |  
---+---+---
 X |   |  
---+---+---
 O |   |  


Choose your move (1-9): 6


 X | O |  
---+---+---
 X |   | X
---+---+---
 O |   |  



AI is making a move...



 X | O |  
---+---+---
 X | O | X
---+---+---
 O |   |  


Choose your move (1-9): 3


 X | O | X
---+---+---
 X | O | X
---+---+---
 O |   |  



AI is making a move...



 X | O | X
---+---+---
 X | O | X
---+---+---
 O | O |  


AI wins!
Do you want to play again? (yes/no): yes
Choose difficulty (very easy, easy,